# Conception d'une application au service de la santé publique
Source des données : https://world.openfoodfacts.org/

# Nettoyage du jeu de données OpenFood Facts
# Sommaire
## I) Import des données
## II) Fusion des catégories redondantes
## III) Filtrage des valeurs aberrantes
## IV) Imputation des valeurs manquantes        

In [24]:
# Import des librairies utiles à notre nettyage
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import scipy
import sklearn
from sklearn.impute import KNNImputer


In [25]:
# Import d'une partie du jeu de données pour avoir une idée des variables pertinentes à traiter pour l'idée d'application
pd.set_option('display.max_column', None)
off = pd.read_csv('en.openfoodfacts.org.products.csv',sep='\t', 
iterator = True, error_bad_lines = False, low_memory=False)
chunk_sample=off.get_chunk(50000)
chunk_sample

/home/jo/miniconda3/envs/johnny_lerouge/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,brand_owner,ecoscore_score_fr,ecoscore_grade_fr,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,-soluble-fiber_100g,-insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,00000000000000225,http://world-en.openfoodfacts.org/product/0000...,nutrinet-sante,1623855208,2021-06-16T14:53:28Z,1623855209,2021-06-16T14:53:29Z,jeunes pousses,NaN,NaN,NaN,NaN,NaN,NaN,endives,endives,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:france,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

# I) Import des données

In [26]:
# Chargement des données avec les variables utiles pour notre l'application
off = pd.read_csv('en.openfoodfacts.org.products.csv',sep='\t', usecols=['code','product_name','pnns_groups_1','pnns_groups_2','countries_en','nutriscore_score','nutriscore_grade','energy-kcal_100g','energy_100g','fat_100g','saturated-fat_100g','trans-fat_100g','cholesterol_100g','carbohydrates_100g','sugars_100g','fiber_100g','proteins_100g','salt_100g','sodium_100g','nutrition-score-fr_100g','nutrition-score-uk_100g'])
off

/home/jo/miniconda3/envs/johnny_lerouge/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,code,product_name,countries_en,nutriscore_score,nutriscore_grade,pnns_groups_1,pnns_groups_2,energy-kcal_100g,energy_100g,fat_100g,saturated-fat_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
0,00000000000000225,jeunes pousses,France,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000017,Vitória crackers,France,NaN,NaN,unknown,unknown,375.0,1569.0,7.0,3.08,NaN,NaN,70.1,15.0,NaN,7.8,1.40,0.560,NaN,NaN
2,0000000000031,Cacao,France,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000000000003327986,Filetes de pollo empanado,Spain,NaN,NaN,unknown,unknown,163.9,685.8,1.9,1.00,NaN,NaN,NaN,NaN,NaN,15.3,1.10,0.440,NaN,NaN
4,000000000004622327,Hamburguesas de ternera 100%,Spain,NaN,NaN,unknown,unknown,874.9,3661.0,15.1,6.10,NaN,NaN,2.6,1.0,NaN,15.7,2.10,0.840,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1875314,9999999910128,Sandwich club Rillette poisson combava,Réunion,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1875315,999999999,Thé noir BIO Darjeeling,France,NaN,NaN,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1875316,9999999990397,Fati,Belgium,NaN,NaN,unknown,unknown,24.0,100.0,0.3,0.00,NaN,NaN,2.4,0.6,NaN,1.6,0.64,0.256,NaN,NaN
1875317,9999999999994,Light & Free SKYR A BOIRE,France,NaN,NaN,unknown,unknown,0.0,0.0,0.2,0.10,NaN,NaN,8.0,7.8,NaN,5.5,0.15,0.060,NaN,NaN


Les colonnes jugées utiles pour notre idée d'application:

* code: code barre du produit

* product_name: nom du produit

* nutriscore_score, nutriscore_grade: nutriscore du produit (vu que l'application se base en partie sur le nutriscore)

* pnns_groups_1: catégorie de produit alimentaire (ex: snacks sucrés, fruits et légumes)

* pnns_groups_2: sous-catégorie de pnns_groups_1 (ex: chips, pâtisseries, boissons alcoolisées)

* energy-kcal_100g, energy_100g, fat_100g, saturated-fat_100g, trans-fat_100g, cholesterol_100g, carbohydrates_100g, sugars_100g, fiber_100g, proteins_100g, salt_100g, sodium_100g: variables renseignant sur la valeur nutritionnelle des produits alimentaires, elles sont aussi liées au nutriscore qui nous intéresse.

On se retrouve avec un jeu de données de 1875319 entrées et 21 colonnes (au lieu des 186 originelles)

In [27]:
# On renomme nos colonnes en remplaçant les tirets par l'underscore
off.columns=off.columns.str.replace('-','_')

In [28]:
# On ne garde que les produits de type snack dans notre dataset
off=off.loc[(off.pnns_groups_1=='Sugary snacks')|(off.pnns_groups_1=='Salty snacks')]
off

,code,product_name,countries_en,nutriscore_score,nutriscore_grade,pnns_groups_1,pnns_groups_2,energy_kcal_100g,energy_100g,fat_100g,saturated_fat_100g,trans_fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,nutrition_score_fr_100g,nutrition_score_uk_100g
25,0000000001281,Tarte noix de coco,France,14.0,d,Sugary snacks,Biscuits and cakes,381.0,1594.0,22.00,15.500,NaN,NaN,27.300,21.900,4.4,4.60,0.10,0.040,14.0,NaN
77,000000000886,Miel de thym,France,NaN,NaN,Sugary snacks,Sweets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,000000001146,Miel de printemps,France,NaN,NaN,Sugary snacks,Sweets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,000000001231,Rillettes de sanglier au Chouchen,France,21.0,e,Salty snacks,Salty and fatty products,325.0,1360.0,29.00,11.000,NaN,NaN,0.000,0.000,NaN,16.00,1.60,0.640,21.0,NaN
168,000000002218,Rillettes de canard au foie gras,France,14.0,d,Salty snacks,Salty and fatty products,25.0,105.0,42.75,17.275,NaN,NaN,0.475,0.075,NaN,13.05,1.10,0.440,14.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1875282,9999990004529,Miel d'acacia ruchers de badon,France,NaN,NaN,Sugary snacks,Sweets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1875283,9999990005052,Confiture de pêche,France,NaN,NaN,Sugary snacks,Sweets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1875304,9999991950191,Pain d’epices,France,NaN,NaN,Sugary snacks,Biscuits and cakes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1875310,9999999175305,Erdbeerkuchen 1019g tiefgefroren,Germany,12.0,d,Sugary snacks,Biscuits and cakes,NaN,925.0,7.60,4.800,NaN,NaN,35.000,24.000,NaN,2.60,0.28,0.112,12.0,NaN


In [29]:
# On convertit la colonne code en type entier
off['code']=off['code'].astype(float)
off['code']=off['code'].astype(int)
off.code.dtype

/tmp/ipykernel_27073/2799329955.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  off['code']=off['code'].astype(float)
/tmp/ipykernel_27073/2799329955.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  off['code']=off['code'].astype(int)


dtype('int64')

In [30]:
# On élimine les doublons
off=off.drop_duplicates(subset='code', keep='first')
off

,code,product_name,countries_en,nutriscore_score,nutriscore_grade,pnns_groups_1,pnns_groups_2,energy_kcal_100g,energy_100g,fat_100g,saturated_fat_100g,trans_fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,nutrition_score_fr_100g,nutrition_score_uk_100g
25,1281,Tarte noix de coco,France,14.0,d,Sugary snacks,Biscuits and cakes,381.0,1594.0,22.00,15.500,NaN,NaN,27.300,21.900,4.4,4.60,0.10,0.040,14.0,NaN
77,886,Miel de thym,France,NaN,NaN,Sugary snacks,Sweets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,1146,Miel de printemps,France,NaN,NaN,Sugary snacks,Sweets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,1231,Rillettes de sanglier au Chouchen,France,21.0,e,Salty snacks,Salty and fatty products,325.0,1360.0,29.00,11.000,NaN,NaN,0.000,0.000,NaN,16.00,1.60,0.640,21.0,NaN
168,2218,Rillettes de canard au foie gras,France,14.0,d,Salty snacks,Salty and fatty products,25.0,105.0,42.75,17.275,NaN,NaN,0.475,0.075,NaN,13.05,1.10,0.440,14.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1875282,9999990004529,Miel d'acacia ruchers de badon,France,NaN,NaN,Sugary snacks,Sweets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1875283,9999990005052,Confiture de pêche,France,NaN,NaN,Sugary snacks,Sweets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1875304,9999991950191,Pain d’epices,France,NaN,NaN,Sugary snacks,Biscuits and cakes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1875310,9999999175305,Erdbeerkuchen 1019g tiefgefroren,Germany,12.0,d,Sugary snacks,Biscuits and cakes,NaN,925.0,7.60,4.800,NaN,NaN,35.000,24.000,NaN,2.60,0.28,0.112,12.0,NaN


122 doublons dans notre jeu de données ont été filtrés.

In [31]:
# Exploration des variables pnns_groups_1 et pnns_group_2
off.pnns_groups_1.unique()

array(['Sugary snacks', 'Salty snacks'], dtype=object)

In [32]:
off.pnns_groups_2.unique()

array(['Biscuits and cakes', 'Sweets', 'Salty and fatty products', 'Nuts',
       'Appetizers', 'Pastries', 'Chocolate products'], dtype=object)

In [33]:
# Fusion des catégories équivalentes
off.loc[off.pnns_groups_1=='sugary-snacks', 'pnns_groups_1']='Sugary snacks'
off.loc[off.pnns_groups_1=='salty-snacks', 'pnns_groups_1']='Salty snacks'
off.loc[off.pnns_groups_2=='nuts', 'pnns_groups_2']='Nuts'
off.loc[off.pnns_groups_2=='pastries', 'pnns_groups_2']='Pastries'

/home/jo/miniconda3/envs/johnny_lerouge/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [34]:
# On créé une variable "Eco_friendly"
off['Eco_friendly']=off['countries_en']

/tmp/ipykernel_27073/4253967647.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  off['Eco_friendly']=off['countries_en']


In [35]:
off=off[['code', 'product_name', 'countries_en', 'nutriscore_score',
       'nutriscore_grade', 'pnns_groups_1', 'pnns_groups_2','Eco_friendly',
       'energy_kcal_100g', 'energy_100g', 'fat_100g', 'saturated_fat_100g',
       'trans_fat_100g', 'cholesterol_100g', 'carbohydrates_100g',
       'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g',
       'sodium_100g', 'nutrition_score_fr_100g', 'nutrition_score_uk_100g'       ]]

In [36]:
# Dictionnaire des pays producteurs avec leur occurence
dict(off.countries_en.value_counts())

{'France': 77401,
 'United States': 52995,
 'Spain': 12668,
 'Germany': 6094,
 'Italy': 3311,
 'France,Spain': 2554,
 'Switzerland': 2360,
 'Belgium': 2177,
 'United Kingdom': 1782,
 'France,Switzerland': 1533,
 'Belgium,France': 1499,
 'France,United States': 1259,
 'France,Germany': 1085,
 'Mexico': 772,
 'Canada': 753,
 'Australia': 734,
 'France,United Kingdom': 691,
 'France,Italy': 510,
 'Sweden': 454,
 'Netherlands': 442,
 'Argentina': 430,
 'Thailand': 364,
 'Poland': 338,
 'Russia': 300,
 'Germany,Switzerland': 248,
 'Portugal': 231,
 'France,Germany,Switzerland': 229,
 'Morocco': 212,
 'Austria': 208,
 'Ireland': 199,
 'Finland': 190,
 'France,Netherlands': 183,
 'Colombia': 182,
 'Romania': 170,
 'Czech Republic': 167,
 'Norway': 161,
 'France,Réunion': 153,
 'Canada,France': 144,
 'Brazil': 144,
 'Hungary': 138,
 'Australia,France': 129,
 'Algeria': 107,
 'Finland,France': 104,
 'Serbia': 104,
 'Germany,Ireland': 102,
 'Luxembourg': 94,
 'Austria,Germany': 92,
 'Spain,Unite

In [37]:
# Création d'un dictionnaire pour affecter à chaque pays producteur le label éco-friendly ou pas
dico = {'France': True,'United States': False,
 'Spain': True,
 'Germany': True,
 'Italy': True,
 'France,Spain': True,
 'Switzerland': True,
 'Belgium': True,
 'United Kingdom': True,
 'France,Switzerland': True,
 'Belgium,France': True,
 'France,United States': False,
 'France,Germany': True,
 'Mexico': False,
 'Canada': False,
 'Australia': False,
 'France,United Kingdom': True,
 'France,Italy': True,
 'Sweden': True,
 'Netherlands': True,
 'Argentina': False,
 'Thailand': False,
 'Poland': True,
 'Russia': False,
 'Germany,Switzerland': True,
 'Portugal': True,
 'France,Germany,Switzerland': True,
 'Morocco': False,
 'Austria': True,
 'Ireland': True,
 'Finland': True,
 'France,Netherlands': True,
 'Colombia': False,
 'Romania': False,
 'Czech Republic': False,
 'Norway': False,
 'France,Réunion': False,
 'Brazil': False,
 'Canada,France': False,
 'Hungary': False,
 'Australia,France': False,
 'Algeria': False,
 'Serbia': False,
 'Finland,France': True,
 'Germany,Ireland': False,
 'Luxembourg': True,
 'Austria,Germany': True,
 'Spain,United States': False,
 'India': False,
 'United Kingdom,United States': False,
 'Belgium,France,Switzerland': True,
 'Bolivia': False,
 'New Caledonia': False,
 'France,Morocco': False,
 'France,Portugal': True,
 'Tunisia': False,
 'Denmark': True,
 'Austria,France,Germany': True,
 'Singapore': False,
 'France,Germany,Spain': True,
 'Japan': False,
 'Germany,Spain': True,
 'Austria,France': True,
 'Hong Kong': False,
 'Bulgaria': False,
 'Chile': False,
 'Germany,United States': False,
 'French Polynesia': False,
 'Portugal,Spain': True,
 'Madagascar': False,
 'France,Spain,Switzerland': True,
 'France,Sweden': True,
 'France,Portugal,Spain': True,
 'Réunion': False,
 'Mexico,United States': False,
 'France,Italy,Spain': True,
 'New Zealand': False,
 'France,Guadeloupe': False,
 'Greece': True,
 'France,Ireland': True,
 'France,French Polynesia': False,
 'South Africa': False,
 'Turkey': False,
 'France,Poland': True,
 'France,Italy,Switzerland': True,
 'Canada,United States': False,
 'Algeria,France': False,
 'France,Tunisia': False,
 'France,Germany,Italy': True,
 'Belgium,France,Spain': True,
 'Belgium,Spain' : True,
 'United Arab Emirates': False,
 'France,United Kingdom,United States': False,
 'Belgium,Germany': True,
 'Belgium,France,Trueetherlands': True,
 'Belgium,France,Germany': True,
 'France,Luxembourg': True,
 'Mexico,Spain': False,
 'Italy,Switzerland': True,
 'Belgium,France,Luxembourg' : True,
 'Czech Republic,France': True,
 'Argentina,Spain': False,
 'Latvia': False,
 'Italy,Spain': True,
 'France,Martinique': True,
 'Guadeloupe': False,
 'France,Mexico': False,
 'Senegal': False,
 'France,Thailand': False,
 'Germany,Luxembourg': True,
 'France,Spain,United States': False,
 'France,Germany,United States': False,
 'Spain,Francia': True,
 'Belgium,Netherlands': True,
 'Spain,United Kingdom': True,
 'Austria,France,Germany,Switzerland': True,
 "Côte d'Ivoire": False,
 'Denmark,France': True,
 'France,Japan': False,
 'Costa Rica': False,
 'France,Hong Kong': False,
 'Peru': False,
 'France,Russia': False,
 'France,South Africa': False,
 'Germany,Italy': True,
 'Germany,Netherlands': True,
 'France,Germany,Netherlands': True,
 'France,Hungary': True,
 'Saudi Arabia': False,
 'Austria,Czech Republic,France,Germany': True,
 'France,Morocco,Spain': False,
 'France,Spain,United Kingdom': True,
 'China': False,
 'Ukraine': False,
 'France,Ireland,United Kingdom': True,
 'Belgium,United States': False,
 'France,New Caledonia': False,
 'Germany,United Kingdom': True,
 'Belgium,France,Italy': True,
 'Georgia': False,
 'Egypt': False,
 'France,Romania': True,
 'Austria,Germany,Switzerland': True,
 'Croatia': True,
 'France,Singapore': False,
 'Malaysia': False,
 'Martinique': False,
 'Morocco,Spain': False,
 'France,Norway': False,
 'French Polynesia,United States': False,
 'France,Serbia': False,
 'Poland,United States': False,
 'Ireland,United Kingdom': True,
 'France,Switzerland,United States': False,
 'Spain,Switzerland': True,
 'Switzerland,United States': False,
 'Slovenia': True,
 'France,Greece': True,
 'France,Germany,Spain,Switzerland': True,
 'Australia,New Zealand': False,
 'France,Germany,United Kingdom': True,
 'Indonesia': False,
 'Burkina Faso': False,
 'France,French Guiana': False,
 'Lebanon': False,
 'Israel': False,
 'Austria,Switzerland': True,
 'Austria,France,Switzerland': True,
 'Finland,France,Germany': True,
 'Belgium,France,Germany,Switzerland': True,
 'Qatar': False,
 'Belarus': False,
 'France,Netherlands,Spain': True,
 'Australia,United States': False,
 'Andorra,Spain': True,
 'Mauritius': False,
 'France,Switzerland,United Kingdom': True,
 'Belgium,France,Portugal': True,
 'Republic-of-macedonia': True,
 'France,Turkey': False,
 'Austria,Belgium,France,Germany,Switzerland': True,
 'Belgium,Luxembourg': True,
 'Finland,Spain': True,
 'Argentina,France': False,
 'Finland,Sweden': True,
 'Belgium,France,Réunion,Switzerland': False,
 'Spain,fr:francia': True,
 'Belgium,Switzerland': True,
 'Belgium,France,Portugal,Switzerland': True,
 'Belgium,France,Réunion': False,
 'Slovakia': False,
 'Andorra': False,
 'Italy,United States': False,
 'Czech Republic,Slovakia': True,
 'Brazil,France': False,
 'Austria,France,United States': False,
 'Belgium,France,Spain,Switzerland': True,
 'France,Réunion,Switzerland': False,
 'Uruguay': False,
 'Vietnam': False,
 'Belgium,France,United States': False,
 'Pakistan': False,
 'Philippines': 10,
 'France,Germany,Luxembourg': True,
 'Czech Republic,Germany': True,
 'Azerbaijan,Malta': False,
 'Denmark,Germany': True,
 'France,Germany,Italy,Switzerland': True,
 'Iraq': False,
 'Argentina,France,Spain': False,
 'Saint Pierre and Miquelon': False,
 'Afghanistan,France': False,
 'Ireland,United States': False,
 'France,Senegal': False,
 'Italy,fr:francia': True,
 'Hungary,Switzerland': False,
 'Switzerland,United Kingdom': True,
 'Belgium,France,United Kingdom': True,
 'Austria,United States': False,
 'Denmark,Sweden': True,
 'Iceland': False,
 'France,New Zealand': False,
 'Bulgaria,France': True,
 'Deutschland': True,
 'Iran': False,
 'Finland,France,Spain': True,
 'Morocco,Spain,fr:francia': False,
 'Bahrain': False,
 'Taiwan': False,
 'Australia,France,Germany': False,
 'Czech Republic,Hungary,Poland,Slovakia': True,
 'France,Guadeloupe,Martinique': False,
 'South Korea': False,
 'France,Italy,Spain,Switzerland': True,
 'Czech Republic,France,Germany': True,
 'Germany,fr:frankreich': True,
 'Netherlands,Spain': True,
 'Jersey': False,
 'Kuwait': False,
 'El Salvador': False,
 'Australia,France,United States': False,
 'France,Netherlands,Switzerland': True,
 'Ecuador': False,
 'French Guiana': False,
 'France,Germany,Hungary': True,
 'China,France': False,
 'Austria,France,Germany,Spain': True,
 'France,Mexico,Spain': False,
 'Colombia,France': False,
 'Germany,Frankreich': True,
 'Cyprus': False,
 'Jordan': False,
 'France,United Arab Emirates': False,
 'Belgium,China,France': False,
 'Estonia': False,
 'Belgium,France,Germany,Spain': True,
 'Lithuania': True,
 'Democratic Republic of the Congo': False,
 'France,Madagascar': False,
 'France,Belgique': True,
 'Belgium,France,Germany,Netherlands': True,
 'Belgium,France,Serbia': True,
 'Belgium,France,Netherlands':True}

In [38]:
# Remplacement des valeurs pays producteurs par un booléen ("True" si le pays est proche géographiquement,
# "False" sinon)
off['Eco_friendly']=off['Eco_friendly'].replace(dico)

In [39]:
# Création d'une liste qui nous sert à attribuer aux pays producteurs incertains la valeur "False"
off['Eco_friendly']=off['Eco_friendly'].astype(str)
liste=list(off.Eco_friendly)
for i in range(0,len(liste)):
    if liste[i]!='True':
        liste[i]='False'
off['Eco_friendly'] = liste

In [40]:
num_variables=['energy_kcal_100g', 'energy_100g', 'fat_100g', 'saturated_fat_100g',
       'trans_fat_100g', 'cholesterol_100g', 'carbohydrates_100g',
       'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g',
       'sodium_100g', 'nutrition_score_fr_100g']

In [41]:
off.isna().mean()

code                       0.000000
product_name               0.006061
countries_en               0.001334
nutriscore_score           0.189319
nutriscore_grade           0.189319
pnns_groups_1              0.000000
pnns_groups_2              0.000000
Eco_friendly               0.000000
energy_kcal_100g           0.148823
energy_100g                0.096206
fat_100g                   0.095541
saturated_fat_100g         0.136059
trans_fat_100g             0.751399
cholesterol_100g           0.753213
carbohydrates_100g         0.097431
sugars_100g                0.100371
fiber_100g                 0.537871
proteins_100g              0.095405
salt_100g                  0.109378
sodium_100g                0.109383
nutrition_score_fr_100g    0.189302
nutrition_score_uk_100g    0.999984
dtype: float64

In [44]:
# On filtre les colonnes qui ont plus de la moité de valeurs manquantes
off=off.loc[:,off.isna().mean()<0.6]
off

,code,product_name,countries_en,nutriscore_score,nutriscore_grade,pnns_groups_1,pnns_groups_2,Eco_friendly,energy_kcal_100g,energy_100g,fat_100g,saturated_fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,nutrition_score_fr_100g
25,1281,Tarte noix de coco,France,14.0,d,Sugary snacks,Biscuits and cakes,True,381.0,1594.0,22.00,15.500,27.300,21.900,4.4,4.60,0.10,0.040,14.0
77,886,Miel de thym,France,NaN,NaN,Sugary snacks,Sweets,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,1146,Miel de printemps,France,NaN,NaN,Sugary snacks,Sweets,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,1231,Rillettes de sanglier au Chouchen,France,21.0,e,Salty snacks,Salty and fatty products,True,325.0,1360.0,29.00,11.000,0.000,0.000,NaN,16.00,1.60,0.640,21.0
168,2218,Rillettes de canard au foie gras,France,14.0,d,Salty snacks,Salty and fatty products,True,25.0,105.0,42.75,17.275,0.475,0.075,NaN,13.05,1.10,0.440,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1875282,9999990004529,Miel d'acacia ruchers de badon,France,NaN,NaN,Sugary snacks,Sweets,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1875283,9999990005052,Confiture de pêche,France,NaN,NaN,Sugary snacks,Sweets,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1875304,9999991950191,Pain d’epices,France,NaN,NaN,Sugary snacks,Biscuits and cakes,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1875310,9999999175305,Erdbeerkuchen 1019g tiefgefroren,Germany,12.0,d,Sugary snacks,Biscuits and cakes,True,NaN,925.0,7.60,4.800,35.000,24.000,NaN,2.60,0.28,0.112,12.0


Les variables trans_fat_100g, cholesterol_100g ainsi que nutrition_score_uk_100g ont été filtrées car contiennent trop de valeurs manquantes

In [20]:
# Détection des valeurs aberrantes grâce à la fonction describe
off.describe()

,code,nutriscore_score,energy_kcal_100g,energy_100g,fat_100g,saturated_fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,nutrition_score_fr_100g
count,1.836480e+05,148880.000000,1.563170e+05,1.659800e+05,166102.000000,158661.000000,165755.000000,165215.000000,84869.000000,166127.000000,163561.000000,163560.000000,148883.000000
mean,1.474401e+14,16.219452,4.199570e+02,1.770499e+03,19.682867,8.133037,54.602335,33.291807,3.162260,6.308942,0.987927,0.395154,16.219488
std,3.729922e+16,7.192631,3.442189e+03,1.966702e+04,72.928931,9.582052,472.105285,70.671198,4.123502,179.170926,37.733408,15.092716,7.192608
min,-9.223372e+18,-14.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,-4.000000,-8.330000,0.000000,0.000000,-14.000000
25%,2.030728e+11,12.000000,3.330000e+02,1.393000e+03,4.000000,1.300000,42.860000,10.340000,0.000000,2.380000,0.060000,0.024000,12.000000
50%,3.299430e+12,16.000000,4.240000e+02,1.784000e+03,19.000000,5.700000,56.520000,32.500000,2.300000,5.410000,0.345000,0.138000,16.000000
75%,5.024278e+12,22.000000,5.070000e+02,2.134000e+03,30.000000,13.200000,68.420000,50.000000,4.000000,7.500000,0.900000,0.360000,22.000000
max,7.591204e+18,40.000000,1.360000e+06,8.010000e+06,29000.000000,2000.000000,192000.000000,27000.000000,200.000000,73000.000000,14000.000000,5600.000000,40.000000


In [46]:
# On filtre les valeurs aberrantes (i.e les valeurs qui ne sont comprises entre 0 et 100) 
off=off[off.fat_100g<=100]
off=off[off.saturated_fat_100g<=100]
off=off[(off.carbohydrates_100g<=100)&(off.carbohydrates_100g>=0)]
off=off[(off.sugars_100g<=100)&(off.sugars_100g>=0)]
off=off[(off.fiber_100g<=100)&(off.fiber_100g>=0)]
off=off[(off.proteins_100g<=100)&(off.proteins_100g>=0)]
off=off[off.salt_100g<=100]
off=off[off.sodium_100g<=100]
off

,code,product_name,countries_en,nutriscore_score,nutriscore_grade,pnns_groups_1,pnns_groups_2,Eco_friendly,energy_kcal_100g,energy_100g,fat_100g,saturated_fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,nutrition_score_fr_100g
25,1281,Tarte noix de coco,France,14.0,d,Sugary snacks,Biscuits and cakes,True,381.0,1594.0,22.00,15.50,27.30,21.90,4.40,4.60,0.1000,0.04000,14.0
404,5018,Crème d'Artichaut Bio Kazidomi,"Belgium,France",11.0,d,Salty snacks,Salty and fatty products,True,268.0,1104.0,27.40,3.90,1.80,0.00,3.50,1.90,2.7000,1.08000,11.0
461,10206515,Pâte à Sucre,France,18.0,d,Sugary snacks,Sweets,True,411.0,1720.0,6.42,3.53,87.86,81.67,0.00,0.03,0.1000,0.04000,18.0
495,12167005,Brioches roulées avec raisins,Canada,0.0,b,Sugary snacks,Biscuits and cakes,False,128.0,536.0,6.00,1.00,50.00,19.00,1.00,7.00,0.0000,0.00000,0.0
521,20045791,Muffin,United States,17.0,d,Sugary snacks,Biscuits and cakes,False,418.0,1749.0,21.76,5.29,51.76,32.94,2.40,5.88,0.6731,0.26924,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1872581,96137062,Eat Natural Bars With Benefits Highin Fibre,"France,United Kingdom",22.0,e,Sugary snacks,Biscuits and cakes,True,964.0,4040.0,21.60,32.70,55.60,32.40,7.78,4.67,0.0000,0.00000,22.0
1873064,9753185833239,Mélange à cuisson pour brownies,France,NaN,NaN,Sugary snacks,Biscuits and cakes,True,393.0,1663.0,5.10,3.00,80.50,53.50,4.40,4.10,0.1900,0.07600,NaN
1874575,9900000000226,Noisettes,France,-4.0,a,Salty snacks,Nuts,True,NaN,2629.0,60.75,4.46,16.70,4.34,9.70,14.95,0.0000,0.00000,-4.0
1874822,990530101017343360,Barre bio,France,9.0,c,Sugary snacks,Biscuits and cakes,True,509.0,2130.0,32.10,2.80,37.80,29.20,6.40,14.00,0.1000,0.04000,9.0


Le filtrage des valeurs aberrantes a fait passer le jeu de données de 183648 entrées à 82672.

In [22]:
# Taux de valeurs manquantes après le filtrage
off.isna().sum()/len(off.index)

code                       0.000000
product_name               0.003387
countries_en               0.001004
nutriscore_score           0.039844
nutriscore_grade           0.039844
pnns_groups_1              0.000000
pnns_groups_2              0.000000
Eco_friendly               0.000000
energy_kcal_100g           0.064290
energy_100g                0.000629
fat_100g                   0.000000
saturated_fat_100g         0.000000
carbohydrates_100g         0.000000
sugars_100g                0.000000
fiber_100g                 0.000000
proteins_100g              0.000000
salt_100g                  0.000000
sodium_100g                0.000000
nutrition_score_fr_100g    0.039820
dtype: float64

## IV) Imputation des valeurs manquantes        

In [23]:
off.isna().sum()

code                          0
product_name                280
countries_en                 83
nutriscore_score           3294
nutriscore_grade           3294
pnns_groups_1                 0
pnns_groups_2                 0
Eco_friendly                  0
energy_kcal_100g           5315
energy_100g                  52
fat_100g                      0
saturated_fat_100g            0
carbohydrates_100g            0
sugars_100g                   0
fiber_100g                    0
proteins_100g                 0
salt_100g                     0
sodium_100g                   0
nutrition_score_fr_100g    3292
dtype: int64

In [24]:
# On isole les variables quantitatives dans un nouveau dataset off2 pour l'imputation 
off2=off.loc[:,"energy_kcal_100g": "nutrition_score_fr_100g"]
off2

,energy_kcal_100g,energy_100g,fat_100g,saturated_fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,nutrition_score_fr_100g
25,381.0,1594.0,22.00,15.50,27.30,21.90,4.40,4.60,0.1000,0.04000,14.0
404,268.0,1104.0,27.40,3.90,1.80,0.00,3.50,1.90,2.7000,1.08000,11.0
461,411.0,1720.0,6.42,3.53,87.86,81.67,0.00,0.03,0.1000,0.04000,18.0
495,128.0,536.0,6.00,1.00,50.00,19.00,1.00,7.00,0.0000,0.00000,0.0
521,418.0,1749.0,21.76,5.29,51.76,32.94,2.40,5.88,0.6731,0.26924,17.0
...,...,...,...,...,...,...,...,...,...,...,...
1872581,964.0,4040.0,21.60,32.70,55.60,32.40,7.78,4.67,0.0000,0.00000,22.0
1873064,393.0,1663.0,5.10,3.00,80.50,53.50,4.40,4.10,0.1900,0.07600,NaN
1874575,NaN,2629.0,60.75,4.46,16.70,4.34,9.70,14.95,0.0000,0.00000,-4.0
1874822,509.0,2130.0,32.10,2.80,37.80,29.20,6.40,14.00,0.1000,0.04000,9.0


In [25]:
off2.describe()

,energy_kcal_100g,energy_100g,fat_100g,saturated_fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,nutrition_score_fr_100g
count,77357.000000,82620.000000,82672.000000,82672.000000,82672.000000,82672.000000,82672.000000,82672.000000,82672.000000,82672.000000,79380.000000
mean,426.035810,1780.281267,19.515721,7.911746,56.947930,33.100080,3.146826,5.996458,0.688516,0.275415,15.724288
std,125.962953,533.008689,14.316539,7.701617,20.830771,22.311229,3.919877,5.137056,1.422198,0.568875,7.272635
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-14.000000
25%,364.000000,1519.000000,8.850000,1.610000,48.000000,15.700000,0.000000,3.120000,0.107000,0.042800,11.000000
50%,437.000000,1837.000000,19.050000,5.800000,59.520000,33.100000,2.330000,5.400000,0.460000,0.184000,16.000000
75%,500.000000,2105.000000,28.120000,12.500000,70.000000,48.280000,4.000000,7.320000,0.919125,0.367850,21.000000
max,4614.000000,25200.000000,100.000000,100.000000,100.000000,100.000000,100.000000,97.000000,100.000000,40.000000,36.000000


In [26]:
# Imputation des valeurs manquantes à l'aide d'Iterative_Imputer 
KNNimp = KNNImputer(n_neighbors=2)
imp_off=pd.DataFrame(KNNimp.fit_transform(off2), columns=off2.columns)


In [27]:
imp_off.describe()

,energy_kcal_100g,energy_100g,fat_100g,saturated_fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,nutrition_score_fr_100g
count,82672.000000,82672.000000,82672.000000,82672.000000,82672.000000,82672.000000,82672.000000,82672.000000,82672.000000,82672.000000,82672.000000
mean,426.300100,1780.137908,19.515721,7.911746,56.947930,33.100080,3.146826,5.996458,0.688516,0.275415,15.573126
std,127.435681,533.123815,14.316539,7.701617,20.830771,22.311229,3.919877,5.137056,1.422198,0.568875,7.288597
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-14.000000
25%,364.000000,1519.000000,8.850000,1.610000,48.000000,15.700000,0.000000,3.120000,0.107000,0.042800,11.000000
50%,439.000000,1837.000000,19.050000,5.800000,59.520000,33.100000,2.330000,5.400000,0.460000,0.184000,16.000000
75%,503.000000,2105.000000,28.120000,12.500000,70.000000,48.280000,4.000000,7.320000,0.919125,0.367850,21.000000
max,4614.000000,25200.000000,100.000000,100.000000,100.000000,100.000000,100.000000,97.000000,100.000000,40.000000,36.000000


In [28]:
# Différence avant après imputation off2/imp_off à l'aide de la fonction compare
off2.describe().compare(imp_off.describe())

energy_kcal_100g                 energy_100g                \
                  self         other          self         other   
count     77357.000000  82672.000000  82620.000000  82672.000000   
mean        426.035810    426.300100   1780.281267   1780.137908   
std         125.962953    127.435681    533.008689    533.123815   
50%         437.000000    439.000000           NaN           NaN   
75%         500.000000    503.000000           NaN           NaN   

      nutrition_score_fr_100g                
                         self         other  
count            79380.000000  82672.000000  
mean                15.724288     15.573126  
std                  7.272635      7.288597  
50%                       NaN           NaN  
75%                       NaN           NaN

Après l'imputation, les changements en terme de moyenne, d'écart type et de médiane sont marginaux. On en conclut que l'impact de l'imputation sur la suite de notre analyse est moindre. 

In [29]:
off2 = imp_off

In [30]:
# On arrondit les valeurs de nos variables quantitatives 
off2=off2.apply(lambda x:round(x,2))
off2

,energy_kcal_100g,energy_100g,fat_100g,saturated_fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,nutrition_score_fr_100g
0,381.0,1594.0,22.00,15.50,27.30,21.90,4.40,4.60,0.10,0.04,14.0
1,268.0,1104.0,27.40,3.90,1.80,0.00,3.50,1.90,2.70,1.08,11.0
2,411.0,1720.0,6.42,3.53,87.86,81.67,0.00,0.03,0.10,0.04,18.0
3,128.0,536.0,6.00,1.00,50.00,19.00,1.00,7.00,0.00,0.00,0.0
4,418.0,1749.0,21.76,5.29,51.76,32.94,2.40,5.88,0.67,0.27,17.0
...,...,...,...,...,...,...,...,...,...,...,...
82667,964.0,4040.0,21.60,32.70,55.60,32.40,7.78,4.67,0.00,0.00,22.0
82668,393.0,1663.0,5.10,3.00,80.50,53.50,4.40,4.10,0.19,0.08,13.5
82669,628.5,2629.0,60.75,4.46,16.70,4.34,9.70,14.95,0.00,0.00,-4.0
82670,509.0,2130.0,32.10,2.80,37.80,29.20,6.40,14.00,0.10,0.04,9.0


In [31]:
# Remise à zéro de l'index d'off pour rendre possible la concaténation
off.reset_index(drop=True, inplace=True)
off

,code,product_name,countries_en,nutriscore_score,nutriscore_grade,pnns_groups_1,pnns_groups_2,Eco_friendly,energy_kcal_100g,energy_100g,fat_100g,saturated_fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,nutrition_score_fr_100g
0,1281,Tarte noix de coco,France,14.0,d,Sugary snacks,Biscuits and cakes,True,381.0,1594.0,22.00,15.50,27.30,21.90,4.40,4.60,0.1000,0.04000,14.0
1,5018,Crème d'Artichaut Bio Kazidomi,"Belgium,France",11.0,d,Salty snacks,Salty and fatty products,True,268.0,1104.0,27.40,3.90,1.80,0.00,3.50,1.90,2.7000,1.08000,11.0
2,10206515,Pâte à Sucre,France,18.0,d,Sugary snacks,Sweets,True,411.0,1720.0,6.42,3.53,87.86,81.67,0.00,0.03,0.1000,0.04000,18.0
3,12167005,Brioches roulées avec raisins,Canada,0.0,b,Sugary snacks,Biscuits and cakes,False,128.0,536.0,6.00,1.00,50.00,19.00,1.00,7.00,0.0000,0.00000,0.0
4,20045791,Muffin,United States,17.0,d,Sugary snacks,Biscuits and cakes,False,418.0,1749.0,21.76,5.29,51.76,32.94,2.40,5.88,0.6731,0.26924,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82667,96137062,Eat Natural Bars With Benefits Highin Fibre,"France,United Kingdom",22.0,e,Sugary snacks,Biscuits and cakes,True,964.0,4040.0,21.60,32.70,55.60,32.40,7.78,4.67,0.0000,0.00000,22.0
82668,9753185833239,Mélange à cuisson pour brownies,France,NaN,NaN,Sugary snacks,Biscuits and cakes,True,393.0,1663.0,5.10,3.00,80.50,53.50,4.40,4.10,0.1900,0.07600,NaN
82669,9900000000226,Noisettes,France,-4.0,a,Salty snacks,Nuts,True,NaN,2629.0,60.75,4.46,16.70,4.34,9.70,14.95,0.0000,0.00000,-4.0
82670,990530101017343360,Barre bio,France,9.0,c,Sugary snacks,Biscuits and cakes,True,509.0,2130.0,32.10,2.80,37.80,29.20,6.40,14.00,0.1000,0.04000,9.0


In [32]:
# On concatène off et off2 dont on a imputé les valeurs manquantes
off=pd.concat([off.loc[:,"code": "Eco_friendly"],off2], axis=1)
off

,code,product_name,countries_en,nutriscore_score,nutriscore_grade,pnns_groups_1,pnns_groups_2,Eco_friendly,energy_kcal_100g,energy_100g,fat_100g,saturated_fat_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,nutrition_score_fr_100g
0,1281,Tarte noix de coco,France,14.0,d,Sugary snacks,Biscuits and cakes,True,381.0,1594.0,22.00,15.50,27.30,21.90,4.40,4.60,0.10,0.04,14.0
1,5018,Crème d'Artichaut Bio Kazidomi,"Belgium,France",11.0,d,Salty snacks,Salty and fatty products,True,268.0,1104.0,27.40,3.90,1.80,0.00,3.50,1.90,2.70,1.08,11.0
2,10206515,Pâte à Sucre,France,18.0,d,Sugary snacks,Sweets,True,411.0,1720.0,6.42,3.53,87.86,81.67,0.00,0.03,0.10,0.04,18.0
3,12167005,Brioches roulées avec raisins,Canada,0.0,b,Sugary snacks,Biscuits and cakes,False,128.0,536.0,6.00,1.00,50.00,19.00,1.00,7.00,0.00,0.00,0.0
4,20045791,Muffin,United States,17.0,d,Sugary snacks,Biscuits and cakes,False,418.0,1749.0,21.76,5.29,51.76,32.94,2.40,5.88,0.67,0.27,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82667,96137062,Eat Natural Bars With Benefits Highin Fibre,"France,United Kingdom",22.0,e,Sugary snacks,Biscuits and cakes,True,964.0,4040.0,21.60,32.70,55.60,32.40,7.78,4.67,0.00,0.00,22.0
82668,9753185833239,Mélange à cuisson pour brownies,France,NaN,NaN,Sugary snacks,Biscuits and cakes,True,393.0,1663.0,5.10,3.00,80.50,53.50,4.40,4.10,0.19,0.08,13.5
82669,9900000000226,Noisettes,France,-4.0,a,Salty snacks,Nuts,True,628.5,2629.0,60.75,4.46,16.70,4.34,9.70,14.95,0.00,0.00,-4.0
82670,990530101017343360,Barre bio,France,9.0,c,Sugary snacks,Biscuits and cakes,True,509.0,2130.0,32.10,2.80,37.80,29.20,6.40,14.00,0.10,0.04,9.0


Notre jeu de données initial contenait 1875319 entrées. Après l'opération de nettoyage, il n'en compte plus que 82672.

In [33]:
# On fait une copie de notre jeu de données au format csv pour une utilisation ultérieure
off.to_csv('off.csv', index=False)